In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input


import numpy as np
import pandas as pd

2024-10-19 19:29:10.512198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 19:29:10.521188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 19:29:10.531042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 19:29:10.534399: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 19:29:10.542788: I tensorflow/core/platform/cpu_feature_guar

In [2]:
imsize = 224
test_dir = '../test/'
test_data = pd.read_csv('../test.csv')
test_data['filename'] = test_data['id'].apply(lambda x: f"{test_dir}{x}.jpg")

In [3]:
test_ds = tf.data.Dataset.from_tensor_slices((test_data['filename']))
def preprocess(filepath):
    image = tf.io.read_file(filepath) # load file
    image = tf.image.decode_jpeg(image, channels=3) # decode it into a tensor
    image = tf.image.resize(image, [224, 224])
    image = (image - 127.5) / 127.5
    return image
test_ds = test_ds.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

I0000 00:00:1729326551.721522  648899 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729326551.736273  648899 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729326551.736315  648899 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729326551.738020  648899 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729326551.738059  648899 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [4]:
resnet_baseline = load_model("../model/resnet152.keras")

In [5]:
predictions = resnet_baseline.predict(test_ds)

I0000 00:00:1729326558.902900  648960 service.cc:146] XLA service 0x7f06940042a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729326558.902924  648960 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-10-19 19:29:19.025542: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-19 19:29:19.783054: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-19 19:29:20.415305: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4873', 196 bytes spill stores, 196 bytes spill loads



 4/60 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step

I0000 00:00:1729326562.642669  648960 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step


In [6]:
predicted_labels = np.argmax(predictions, axis=1)

In [7]:
test_data['stable_height'] = predicted_labels
test_data['stable_height'] += 1
test_data[['id', 'stable_height']].to_csv('../predictions/resnet152.csv', index=False)